In [1]:
import $ivy.`org.clulab::processors-main:6.1.3`
import $ivy.`org.clulab::processors-corenlp:6.1.3`
import $ivy.`org.clulab::processors-odin:6.1.3`
import $ivy.`org.clulab::processors-modelsmain:6.1.3`
import $ivy.`org.clulab::processors-modelscorenlp:6.1.3`
import $ivy.`com.lihaoyi::pprint:0.5.3`

import org.clulab.odin.ExtractorEngine
import org.clulab.processors.Processor
import org.clulab.processors.clu.CluProcessor
import org.clulab.processors.corenlp.CoreNLPProcessor
import org.clulab.processors.fastnlp.FastNLPProcessor
import org.clulab.processors.shallownlp.ShallowNLPProcessor
import pprint.pprintln

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                                           

import $ivy.$                          


import org.clulab.odin.ExtractorEngine

import org.clulab.processors.Processor

import org.clulab.processors.clu.CluProcessor

import org.clulab.processors.corenlp.CoreNLPProcessor

import org.clulab.processors.fastnlp.FastNLPProcessor

import org.clulab.processors.shallownlp.ShallowNLPProcessor

import pprint.pprintln

In [11]:
// Stanford one
val proc: Processor = new CoreNLPProcessor()
// Also Stanford but Neural Net based (faster).
// val proc: Processor = new FastNLPProcessor()
// CLU lab implementation (faster but less featureful)
// val proc: Processor = new CluProcessor()
val testPhrase = "Hello the world is not flat."
val doc = proc.annotate(testPhrase, true)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator ner
Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... done [0.3 sec].
macro=true
featureCountThresh=10
featureFactory=org.clulab.processors.corenlp.chunker.ChunkingFeatureFactory
Adding annotator dcoref


proc: Processor = org.clulab.processors.corenlp.CoreNLPProcessor@17184f82
testPhrase: String = "Hello the world is not flat."
doc: org.clulab.processors.Document = org.clulab.processors.corenlp.CoreNLPDocument@4ba1179e

In [12]:

import org.clulab.odin.{EventMention, Mention, RelationMention, TextBoundMention}
import org.clulab.processors.{Document, Sentence}

object NLPPrinter {

  def displayMentions(mentions: Seq[Mention], doc: Document): Unit = {
    val mentionsBySentence = mentions groupBy (_.sentence) mapValues (_.sortBy(_.start)) withDefaultValue Nil
    for ((s, i) <- doc.sentences.zipWithIndex) {
      println(s"sentence #$i")
      println(s.getSentenceText)
      println("Tokens: " + (s.words.indices, s.words, s.tags.get).zipped.mkString(", "))
      printSyntacticDependencies(s)
      println

      val sortedMentions     = mentionsBySentence(i).sortBy(_.label)
      val (events, entities) = sortedMentions.partition(_ matches "Event")
      val (tbs, rels)        = entities.partition(_.isInstanceOf[TextBoundMention])
      val sortedEntities     = tbs ++ rels.sortBy(_.label)
      println("entities:")
      sortedEntities foreach displayMention

      println
      println("events:")
      events foreach displayMention
      println("=" * 50)
    }
  }

  def printSyntacticDependencies(s: Sentence): Unit = {
    if (s.dependencies.isDefined) {
      println(s.dependencies.get.toString)
    }
  }

  def displayMention(mention: Mention) {
    val boundary = s"\t${"-" * 30}"
    println(s"${mention.labels} => ${mention.text}")
    println(boundary)
    println(s"\tRule => ${mention.foundBy}")
    val mentionType = mention.getClass.toString.split("""\.""").last
    println(s"\tType => $mentionType")
    println(boundary)
    mention match {
      case tb: TextBoundMention =>
        println(s"\t${tb.labels.mkString(", ")} => ${tb.text}")
      case em: EventMention =>
        println(s"\ttrigger => ${em.trigger.text}")
        displayArguments(em)
      case rel: RelationMention =>
        displayArguments(rel)
      case _ => ()
    }
    println(s"$boundary\n")
  }

  def displayArguments(b: Mention): Unit = {
    b.arguments foreach {
      case (argName, ms) =>
        ms foreach { v =>
          println(s"\t$argName ${v.labels.mkString("(", ", ", ")")} => ${v.text}")
        }
    }
  }
}


import org.clulab.odin.{EventMention, Mention, RelationMention, TextBoundMention}

import org.clulab.processors.{Document, Sentence}


defined object NLPPrinter

In [15]:
val rules = """
     rules:
       - name: "obj"
         label: Object
         priority: 1
         type: token
         unit: "tag"
         pattern: |
           /^N/* [incoming=/dobj/] /^N/*
     
     
     
""".stripMargin
// Instantiate the yaml with our rules above.
val engine = ExtractorEngine(rules)
// Let's see if this works
val phrase = "where do I go to file a claim?"

val doc = proc.annotate(phrase)
val mentions = engine.extractFrom(doc)
NLPPrinter.displayMentions(mentions, doc)

sentence #0
where do I go to file a claim ?
Tokens: (0,where,WRB), (1,do,VBP), (2,I,PRP), (3,go,VB), (4,to,TO), (5,file,VB), (6,a,DT), (7,claim,NN), (8,?,.)
roots: 3
outgoing:
	0:
	1:
	2:
	3: (0,advmod) (1,aux) (2,nsubj) (5,xcomp)
	4:
	5: (4,aux) (7,dobj)
	6:
	7: (6,det)
incoming:
	0: (3,advmod)
	1: (3,aux)
	2: (3,nsubj)
	3:
	4: (5,aux)
	5: (3,xcomp)
	6: (7,det)
	7: (5,dobj)


entities:
List(Object) => claim
	------------------------------
	Rule => obj
	Type => TextBoundMention
	------------------------------
	Object => claim
	------------------------------


events:


rules: String = """

     rules:
       - name: "obj"
         label: Object
         priority: 1
         type: token
         unit: "tag"
         pattern: |
           /^N/* [incoming=/dobj/] /^N/*
     
     
...
engine: ExtractorEngine = org.clulab.odin.ExtractorEngine@63c51219
phrase: String = "where do I go to file a claim?"
doc: Document = org.clulab.processors.corenlp.CoreNLPDocument@4da510a5
mentions: Seq[Mention] = Vector(org.clulab.odin.TextBoundMention@6e4fc1ad)

In [25]:
val rules = """
     rules:
       - name: "obj"
         label: [Entity, Object]
         priority: 1
         type: token
         unit: "tag"
         pattern: |
           /^N/* [incoming=/dobj/] /^N/*
           
       - name: "ner-location"
         label: Location
         priority: 1
         type: token
         pattern: |
           [entity="LOCATION"]+
           
       - name: "subj"
         label: [Entity, Subject]
         priority: 1
         type: token
         unit: "tag"
         pattern: |
           /^N/* [incoming=/^nsubj/] /^N/*
           
       - name: "ner-date"
         label: Date
         priority: 1
         type: token
         pattern: |
           [entity="DATE"]+
    
       - name: "liability-syntax-1"
         label: liability
         priority: 2
         example: "He sued Fred last june in mexico."
         type: dependency
         pattern: |
           trigger = [tag=/^V/ & lemma="sue"]
           claimaint: Entity+ = <xcomp? /nsubj$/
           defendant: Entity+ = <xcomp? dobj
           location: Location? = /prep_(in|at)/+
     
     
     
""".stripMargin
// Instantiate the yaml with our rules above.
val engine = ExtractorEngine(rules)
// Let's see if this works
val phrase = "he sued Fred last june in Mexico."

val doc = proc.annotate(phrase)
val mentions = engine.extractFrom(doc)
NLPPrinter.displayMentions(mentions, doc)

sentence #0
he sued Fred last june in Mexico .
Tokens: (0,he,PRP), (1,sued,VBD), (2,Fred,NNP), (3,last,JJ), (4,june,NNP), (5,in,IN), (6,Mexico,NNP), (7,.,.)
roots: 1
outgoing:
	0:
	1: (0,nsubj) (2,dobj) (4,tmod) (6,prep_in)
	2:
	3:
	4: (3,amod)
	5:
	6:
incoming:
	0: (1,nsubj)
	1:
	2: (1,dobj)
	3: (4,amod)
	4: (1,tmod)
	5:
	6: (1,prep_in)


entities:
List(Date) => last june
	------------------------------
	Rule => ner-date
	Type => TextBoundMention
	------------------------------
	Date => last june
	------------------------------

List(Entity, Subject) => he
	------------------------------
	Rule => subj
	Type => TextBoundMention
	------------------------------
	Entity, Subject => he
	------------------------------

List(Entity, Object) => Fred
	------------------------------
	Rule => obj
	Type => TextBoundMention
	------------------------------
	Entity, Object => Fred
	------------------------------

List(Location) => Mexico
	------------------------------
	Rule => ner-location
	Type =>

rules: String = """

     rules:
       - name: "obj"
         label: [Entity, Object]
         priority: 1
         type: token
         unit: "tag"
         pattern: |
           /^N/* [incoming=/dobj/] /^N/*
           
       - name: "ner-location"
...
engine: ExtractorEngine = org.clulab.odin.ExtractorEngine@1b0f89a0
phrase: String = "he sued Fred last june in Mexico."
doc: Document = org.clulab.processors.corenlp.CoreNLPDocument@2d31235
mentions: Seq[Mention] = Vector(
  org.clulab.odin.TextBoundMention@5f4443b9,
  org.clulab.odin.EventMention@95a2a5f3,
  org.clulab.odin.TextBoundMention@23773acb,
  org.clulab.odin.TextBoundMention@168345f1,
  org.clulab.odin.TextBoundMention@11895a69
)